** Experiment - 5 **

1. OHE of cities and genres
2. Total watch time
3. Random Forest Model
4. Hyperparameter Tuning

In [16]:
%matplotlib inline

import pandas as pd
import numpy as np
import scipy as sp
import gc
import time

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('dark')

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib

from bayes_opt import BayesianOptimization

pd.set_option('max_columns', None)

SEED = 2131
np.random.seed(SEED)

import warnings
warnings.filterwarnings('ignore')

%run ../src/data/HotstarDataset.py
%run ../src/features/categorical_features.py
%run ../src/features/util.py
%run ../src/models/cross_validation.py

In [2]:
dataset = Hotstar('../data/raw/5f828822-4--4-hotstar_dataset/')
dataset.load_data('../data/processed/hotstar_processed.feather')

In [3]:
data_processed = dataset.data
train_mask     = dataset.get_train_mask() 

In [4]:
data = load_file('../data/processed/ohe_genres.feather')

** Save processed file **

In [38]:
save_file(data_processed.loc[:, ['ID'] + data_processed.columns[7:].tolist()], \
          '../data/processed/ohe_genres.feather')

In [18]:
X = data.loc[train_mask, data.columns.drop('ID', 'Action')]
y = data_processed.loc[train_mask, 'segment']
Xtest = data.loc[~train_mask, data.columns.drop('ID', 'Action')]

** Train, Val, Test Split **

In [6]:
params = {
    'stratify': y,
    'test_size': .3,
    'random_state': SEED
}

X_train, X_test, y_train, y_test = get_train_test_split(X, y, **params)

In [7]:
X_train.shape, X_test.shape

((140000, 35), (60000, 35))

** Model Creation **

In [8]:
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=SEED)
rf.fit(X_train, y_train)
preds = rf.predict_proba(X_test)[:, 1]

print('AUC: {}'.format(roc_auc_score(y_test, preds)))

AUC: 0.7745496076842832


** Hyper-parameter Tuning **

In [9]:
def rfccv(n_estimators, min_samples_split, max_depth, max_features):
    skf = StratifiedKFold(n_splits=3, random_state=SEED)
    val = cross_val_score(
        RandomForestClassifier(n_estimators=int(n_estimators),
                               min_samples_split=int(min_samples_split),
                               max_depth=int(max_depth),
                               max_features=min(max_features, 1),
                               random_state=SEED,
                               class_weight='balanced'
                              ),
        X_train, y_train, scoring='roc_auc', cv=skf
    ).mean()

    return val

def logccv(C):
    skf = StratifiedKFold(n_splits=3, random_state=SEED)

    val = cross_val_score(
        LogisticRegression(C=C,
        n_jobs=2,
        class_weight='balanced',
        random_state=SEED
                          ),
        X_train, y_train, scoring='roc_auc', cv=skf
    ).mean()

    return val

In [10]:
def parameter_search(params, rf):
    gp_params = {
        'alpha': 1e-5
    }

    if rf:
        rfcBO = BayesianOptimization(
            rfccv,
            params
        )
        rfcBO.maximize(n_iter=10, **gp_params)
        print('RFC: %f' % rfcBO.res['max']['max_val'])

    else:
        logcBO = BayesianOptimization(
            logccv,
            {
                'C': (.01, 100)
            }
        )

        logcBO.maximize(n_iter=10, **gp_params)
        print('Logistic Regression: %f' % logcBO.res['max']['max_val'])

In [11]:
rf_params = {
                'n_estimators': (10, 250),
                'min_samples_split': (2, 25),
                'max_depth': (5, 30),
                'max_features': (0.1, 0.99)
            }

parameter_search(rf_params, rf=True)

Initialization
---------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   max_depth |   max_features |   min_samples_split |   n_estimators | 
    1 | 01m17s |    0.77672 |     10.5134 |         0.7362 |              4.3129 |       220.1397 | 
    2 | 01m18s |    0.77911 |      8.4456 |         0.9248 |              5.2475 |       201.2883 | 
    3 | 01m19s |    0.73948 |     28.1288 |         0.5666 |              5.2100 |       189.1249 | 
    4 | 00m11s |    0.77961 |      7.1602 |         0.7216 |             18.0646 |        41.0869 | 
    5 | 00m35s |    0.77974 |     13.9344 |         0.1934 |             13.0437 |       195.9890 | 
Bayesian Optimization
---------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   max_depth |   max_features |   min_samples_split |   n_estimators | 
    6 | 01m17s |    0.77743 |      5.0000 |         0.99

In [12]:
def test_model(X_train, y_train, X_test, y_test, model):
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_test)[:, 1]
    print('AUC on test set: {}'.format(roc_auc_score(y_test, preds)))

In [14]:
# train this model
rf = RandomForestClassifier(n_estimators=248, 
                            class_weight='balanced', 
                            max_depth=5, 
                            min_samples_split=2,
                            max_features=.1703,
                            random_state=SEED,
                            n_jobs=3
                           )

test_model(X_train, y_train, X_test, y_test, rf)

AUC on test set: 0.7741882208888831


In [15]:
def full_training(X, y, Xtest, model, model_name, save=True):
    model.fit(X, y)
    final_preds = model.predict_proba(Xtest)[:, 1]
    
    if save:
        joblib.dump(model, '../models/%s'%(model_name))
        
    return final_preds

In [19]:
final_preds = full_training(X, y, Xtest, rf, 'rf_genres_ohe.pkl')

In [20]:
sub            = pd.read_csv('../data/raw/5f828822-4--4-hotstar_dataset/sample_submission.csv')
sub['segment'] = final_preds
sub.to_csv('../submissions/hotstar/rf_genres_ohe.csv', index=False)